# AustArch Data Visualization

Interactive visualizations for the **AustArch** Australian Archaeological Age Database.

## Dataset Overview

- **~5,200 age determinations** from archaeological sites across Australia
- **~1,100 sites** with geographic coordinates
- **Time span:** 0 to 270,000 years BP (Before Present)
- **Dating methods:** Radiocarbon (C14, AMS, Conventional), OSL, TL, ESR, U-Th

## Data Source

Based on the AustArch dataset: [https://doi.org/10.5284/1027216](https://doi.org/10.5284/1027216)

## Contents

1. [Setup and Database Connection](#1-setup)
2. [Temporal Distribution Chart](#2-temporal-distribution)
3. [Dating Method Breakdown](#3-dating-methods)
4. [State/Territory Coverage](#4-state-coverage)
5. [Material Type Distribution](#5-materials)
6. [Interactive Map](#6-map)
7. [Summary Statistics](#7-summary)

---
<a id="1-setup"></a>
## 1. Setup and Database Connection

### Requirements

```bash
pip install jupyter matplotlib seaborn folium pandas psycopg2-binary sqlalchemy
```

### Database Configuration

Set environment variables or modify `DB_CONFIG` below:

```bash
export AUSTARCH_DB_HOST=localhost
export AUSTARCH_DB_NAME=austarch
export AUSTARCH_DB_USER=your_username
```

In [ ]:
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from sqlalchemy import create_engine

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set style for publication-quality charts
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'figure.dpi': 100,
    'savefig.dpi': 150,
    'savefig.bbox': 'tight'
})

In [ ]:
# Database configuration
DB_CONFIG = {
    'host': os.getenv('AUSTARCH_DB_HOST', 'localhost'),
    'port': os.getenv('AUSTARCH_DB_PORT', '5432'),
    'database': os.getenv('AUSTARCH_DB_NAME', 'austarch'),
    'user': os.getenv('AUSTARCH_DB_USER', os.getenv('USER', 'postgres')),
    'password': os.getenv('AUSTARCH_DB_PASSWORD', '')
}

def get_engine():
    """Create SQLAlchemy database engine."""
    url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    return create_engine(url)

def query_to_df(sql):
    """Execute SQL query and return pandas DataFrame."""
    with get_engine().connect() as conn:
        return pd.read_sql_query(sql, conn)

# Test connection
try:
    engine = get_engine()
    with engine.connect() as conn:
        result = pd.read_sql_query("SELECT COUNT(*) as count FROM age_determination", conn)
        count = result['count'].iloc[0]
    print(f"Connected to database. Found {count:,} age determinations.")
except Exception as e:
    print(f"Connection failed: {e}")
    print("\nPlease check your database configuration:")
    print(f"  Host: {DB_CONFIG['host']}:{DB_CONFIG['port']}")
    print(f"  Database: {DB_CONFIG['database']}")
    print(f"  User: {DB_CONFIG['user']}")

---
<a id="2-temporal-distribution"></a>
## 2. Temporal Distribution

Stacked bar chart showing the distribution of archaeological dates across time, grouped by 1,000-year brackets and colored by dating method.

The **Holocene/Pleistocene boundary** (~11,700 BP) marks the transition from the last ice age to the current warm period.

In [ ]:
# Define consistent color scheme for dating methods
METHOD_COLORS = {
    'C14': '#2E86AB',    # Blue - Radiocarbon
    'AMS': '#1B4F72',    # Dark Blue - AMS Radiocarbon
    'CONV': '#5DADE2',   # Light Blue - Conventional Radiocarbon
    'OSL': '#E94F37',    # Red - Optically Stimulated Luminescence
    'TL': '#F39C12',     # Orange - Thermoluminescence
    'AAR': '#27AE60',    # Green - Amino Acid Racemization
    'ESR': '#8E44AD',    # Purple - Electron Spin Resonance
    'U-TH': '#16A085',   # Teal - Uranium-Thorium
    'OTHER': '#95A5A6',  # Gray - Other methods
}

In [ ]:
# Query temporal distribution data
temporal_sql = """
SELECT 
    (COALESCE(age_bp, c14_age) / 1000) * 1000 AS bracket,
    dm.code AS method, 
    COUNT(*) AS count
FROM age_determination ad
JOIN dating_method dm ON ad.method_id = dm.id
WHERE COALESCE(age_bp, c14_age) IS NOT NULL 
  AND COALESCE(age_bp, c14_age) >= 0
  AND NOT is_rejected
GROUP BY bracket, dm.code 
ORDER BY bracket
"""

temporal_df = query_to_df(temporal_sql)
print(f"Loaded {len(temporal_df)} bracket/method combinations")

In [ ]:
# Pivot data for stacked bar chart
temporal_pivot = temporal_df.pivot(index='bracket', columns='method', values='count').fillna(0)

# Create stacked bar chart
fig, ax = plt.subplots(figsize=(14, 8))

# Get colors for available methods
colors = [METHOD_COLORS.get(col, '#95A5A6') for col in temporal_pivot.columns]
temporal_pivot.plot(kind='bar', stacked=True, ax=ax, color=colors, width=0.85, edgecolor='white', linewidth=0.5)

# Formatting
ax.set_xlabel('Age (years BP)', fontsize=12)
ax.set_ylabel('Number of Dates', fontsize=12)
ax.set_title('Temporal Distribution of Archaeological Dates by Method', fontsize=14, fontweight='bold')

# Format x-axis labels (show every 5th label for readability)
tick_labels = [f"{int(x/1000)}k" if i % 5 == 0 else '' for i, x in enumerate(temporal_pivot.index)]
ax.set_xticklabels(tick_labels, rotation=45, ha='right')

# Add legend
ax.legend(title='Dating Method', loc='upper right', framealpha=0.9)

# Add Holocene/Pleistocene boundary annotation
try:
    boundary_idx = list(temporal_pivot.index).index(11000)
    ax.axvline(x=boundary_idx + 0.5, color='#333333', linestyle='--', alpha=0.7, linewidth=1.5)
    ymax = ax.get_ylim()[1]
    ax.text(boundary_idx + 1, ymax * 0.92, 'Pleistocene', fontsize=10, color='#333333', ha='left', style='italic')
    ax.text(boundary_idx - 0.5, ymax * 0.92, 'Holocene', fontsize=10, color='#333333', ha='right', style='italic')
except ValueError:
    pass  # 11000 bracket not in data

plt.tight_layout()
plt.savefig('temporal_distribution.png')
plt.show()

# Summary statistics
holocene_count = temporal_pivot[temporal_pivot.index < 12000].sum().sum()
total_count = temporal_pivot.sum().sum()
print(f"\nHolocene ages (< 12,000 BP): {holocene_count:,.0f} ({holocene_count/total_count*100:.1f}%)")
print(f"Pleistocene ages (>= 12,000 BP): {total_count - holocene_count:,.0f} ({(1 - holocene_count/total_count)*100:.1f}%)")

---
<a id="3-dating-methods"></a>
## 3. Dating Method Breakdown

Analysis of dating techniques used in the database. **Radiocarbon dating** (C14) is the dominant method for the Holocene period, while **luminescence methods** (OSL, TL) are used for older Pleistocene deposits.

In [ ]:
# Query method breakdown
method_sql = """
SELECT dm.name, dm.code, COUNT(*) as count 
FROM age_determination ad
JOIN dating_method dm ON ad.method_id = dm.id
WHERE NOT is_rejected 
GROUP BY dm.name, dm.code 
ORDER BY count DESC
"""

method_df = query_to_df(method_sql)
method_df

In [ ]:
# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Horizontal bar chart
colors = [METHOD_COLORS.get(code, '#95A5A6') for code in method_df['code']]
bars = ax1.barh(method_df['name'], method_df['count'], color=colors, edgecolor='white', linewidth=0.5)
ax1.set_xlabel('Number of Dates', fontsize=12)
ax1.set_title('Age Determinations by Dating Method', fontsize=14, fontweight='bold')
ax1.invert_yaxis()

# Add count labels on bars
max_count = method_df['count'].max()
for bar, count in zip(bars, method_df['count']):
    ax1.text(bar.get_width() + max_count * 0.02, bar.get_y() + bar.get_height()/2, 
             f'{count:,}', va='center', fontsize=10)

# Pie chart - group small categories
pie_df = method_df.copy()
threshold = method_df['count'].sum() * 0.02
small_methods = pie_df[pie_df['count'] < threshold]
if len(small_methods) > 0:
    other_count = small_methods['count'].sum()
    pie_df = pie_df[pie_df['count'] >= threshold].copy()
    pie_df = pd.concat([pie_df, pd.DataFrame({'name': ['Other'], 'code': ['OTHER'], 'count': [other_count]})])

pie_colors = [METHOD_COLORS.get(code, '#95A5A6') for code in pie_df['code']]
wedges, texts, autotexts = ax2.pie(
    pie_df['count'], 
    labels=pie_df['name'], 
    autopct='%1.1f%%', 
    colors=pie_colors,
    pctdistance=0.75, 
    startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1}
)
ax2.set_title('Proportion of Dating Methods', fontsize=14, fontweight='bold')

for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.savefig('method_breakdown.png')
plt.show()

# Summary
total = method_df['count'].sum()
print(f"\nTotal valid age determinations: {total:,}")
for _, row in method_df.iterrows():
    print(f"  {row['name']}: {row['count']:,} ({row['count']/total*100:.1f}%)")

---
<a id="4-state-coverage"></a>
## 4. State/Territory Coverage

Geographic distribution of archaeological sites and dates across Australian states and territories.

In [ ]:
# Query state coverage
state_sql = """
SELECT 
    state,
    COUNT(DISTINCT s.id) as sites,
    COUNT(ad.id) as dates
FROM site s
LEFT JOIN sample sa ON sa.site_id = s.id
LEFT JOIN age_determination ad ON ad.sample_id = sa.id AND NOT ad.is_rejected
WHERE state IS NOT NULL
GROUP BY state 
ORDER BY sites DESC
"""

state_df = query_to_df(state_sql)
state_df

In [ ]:
# Create grouped horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 8))

x = range(len(state_df))
width = 0.35

bars1 = ax.barh([i - width/2 for i in x], state_df['sites'], width, 
                label='Sites', color='#2E86AB', alpha=0.9, edgecolor='white')
bars2 = ax.barh([i + width/2 for i in x], state_df['dates'], width, 
                label='Dates', color='#E94F37', alpha=0.9, edgecolor='white')

ax.set_yticks(x)
ax.set_yticklabels(state_df['state'])
ax.set_xlabel('Count', fontsize=12)
ax.set_title('Archaeological Coverage by State/Territory', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', framealpha=0.9)
ax.invert_yaxis()

# Add value labels
for bar, sites in zip(bars1, state_df['sites']):
    ax.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2, 
            f'{sites:,}', va='center', fontsize=9, color='#2E86AB', fontweight='bold')
for bar, dates in zip(bars2, state_df['dates']):
    ax.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2, 
            f'{dates:,}', va='center', fontsize=9, color='#E94F37', fontweight='bold')

plt.tight_layout()
plt.savefig('state_coverage.png')
plt.show()

print(f"\nTotal sites: {state_df['sites'].sum():,}")
print(f"Total dates: {state_df['dates'].sum():,}")

---
<a id="5-materials"></a>
## 5. Material Type Distribution

Distribution of sample materials used for dating. **Charcoal** and **shell** are the most common materials for radiocarbon dating, while **quartz** and other minerals are used for luminescence dating.

In [ ]:
# Query material distribution
material_sql = """
SELECT 
    COALESCE(sm.name, 'Unknown') as material,
    COALESCE(sm.category, 'unknown') as category,
    COUNT(*) as count
FROM sample sa
LEFT JOIN sample_material sm ON sa.material_id = sm.id
JOIN age_determination ad ON ad.sample_id = sa.id AND NOT ad.is_rejected
GROUP BY sm.name, sm.category
ORDER BY count DESC
"""

material_df = query_to_df(material_sql)
material_df.head(10)

In [ ]:
# Color scheme for material categories
CATEGORY_COLORS = {
    'organic': '#27AE60',     # Green
    'inorganic': '#E74C3C',   # Red
    'mixed': '#F39C12',       # Orange
    'other': '#9B59B6',       # Purple
    'unknown': '#95A5A6',     # Gray
}

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 10))

# Top 15 materials for readability
plot_df = material_df.head(15).copy()
colors = [CATEGORY_COLORS.get(cat, '#95A5A6') for cat in plot_df['category']]

bars = ax.barh(plot_df['material'], plot_df['count'], color=colors, edgecolor='white', linewidth=0.5)
ax.set_xlabel('Number of Samples', fontsize=12)
ax.set_title('Sample Material Types (Top 15)', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add count labels
max_count = plot_df['count'].max()
for bar, count in zip(bars, plot_df['count']):
    ax.text(bar.get_width() + max_count * 0.01, bar.get_y() + bar.get_height()/2,
            f'{count:,}', va='center', fontsize=10)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=cat.title(), edgecolor='white') 
                   for cat, color in CATEGORY_COLORS.items() 
                   if cat in plot_df['category'].values]
ax.legend(handles=legend_elements, title='Category', loc='lower right', framealpha=0.9)

plt.tight_layout()
plt.savefig('material_distribution.png')
plt.show()

print(f"\nMaterial types represented: {len(material_df)}")
print(f"Total samples with dates: {material_df['count'].sum():,}")

---
<a id="6-map"></a>
## 6. Interactive Map

Clustered marker map showing archaeological site locations across Australia. Click on clusters to zoom in, and click individual markers to view site details.

**Marker colors by site type:**
- Red: Rockshelter/Cave
- Blue: Open site
- Green: Midden/Shell mound
- Purple: Art site
- Orange: Quarry
- Gray: Other/Unknown

In [ ]:
# Query map data
map_sql = """
SELECT 
    s.id,
    s.site_name,
    s.latitude,
    s.longitude,
    s.state,
    s.site_type,
    COUNT(ad.id) as dates,
    MIN(COALESCE(ad.age_bp, ad.c14_age)) as youngest_age,
    MAX(COALESCE(ad.age_bp, ad.c14_age)) as oldest_age
FROM site s
LEFT JOIN sample sa ON sa.site_id = s.id
LEFT JOIN age_determination ad ON ad.sample_id = sa.id AND NOT ad.is_rejected
WHERE s.latitude IS NOT NULL 
  AND s.longitude IS NOT NULL
  AND s.latitude BETWEEN -45 AND -10
  AND s.longitude BETWEEN 110 AND 160
GROUP BY s.id, s.site_name, s.latitude, s.longitude, s.state, s.site_type
ORDER BY dates DESC
"""

map_df = query_to_df(map_sql)
print(f"Sites with valid coordinates: {len(map_df):,}")

In [ ]:
# Site type color mapping
SITE_TYPE_COLORS = {
    'rockshelter': 'red',
    'cave': 'darkred',
    'open': 'blue',
    'midden': 'green',
    'shell mound': 'darkgreen',
    'art': 'purple',
    'quarry': 'orange',
    'burial': 'black',
}

def get_marker_color(site_type):
    """Get marker color based on site type."""
    if not site_type:
        return 'gray'
    site_type_lower = str(site_type).lower()
    for key, color in SITE_TYPE_COLORS.items():
        if key in site_type_lower:
            return color
    return 'gray'

# Create map centered on Australia
m = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles='cartodbpositron'
)

# Create marker cluster
marker_cluster = MarkerCluster(
    name='Archaeological Sites',
    options={
        'maxClusterRadius': 50,
        'spiderfyOnMaxZoom': True,
        'showCoverageOnHover': True,
        'zoomToBoundsOnClick': True
    }
)

# Add markers
for _, row in map_df.iterrows():
    # Build popup HTML
    age_range = ''
    if pd.notna(row['youngest_age']) and pd.notna(row['oldest_age']):
        age_range = f"<tr><td><b>Age Range:</b></td><td>{row['youngest_age']:,.0f} - {row['oldest_age']:,.0f} BP</td></tr>"
    
    popup_html = f"""
    <div style="min-width: 200px; font-family: Arial, sans-serif;">
        <h4 style="margin: 0 0 8px 0; color: #333;">{row['site_name']}</h4>
        <table style="font-size: 12px; border-collapse: collapse;">
            <tr><td style="padding: 2px 8px 2px 0;"><b>State:</b></td><td>{row['state'] or 'Unknown'}</td></tr>
            <tr><td style="padding: 2px 8px 2px 0;"><b>Type:</b></td><td>{row['site_type'] or 'Unknown'}</td></tr>
            <tr><td style="padding: 2px 8px 2px 0;"><b>Dates:</b></td><td>{row['dates']:,}</td></tr>
            {age_range}
        </table>
    </div>
    """
    
    marker = folium.Marker(
        location=[float(row['latitude']), float(row['longitude'])],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=row['site_name'],
        icon=folium.Icon(color=get_marker_color(row['site_type']), icon='info-sign')
    )
    marker.add_to(marker_cluster)

marker_cluster.add_to(m)
folium.LayerControl().add_to(m)

# Save and display
m.save('austarch_map.html')
print("Interactive map saved to: austarch_map.html")
m

---
<a id="7-summary"></a>
## 7. Summary Statistics

In [ ]:
# Overall summary statistics
summary_sql = """
SELECT 
    (SELECT COUNT(*) FROM site) as total_sites,
    (SELECT COUNT(*) FROM site WHERE latitude IS NOT NULL) as sites_with_coords,
    (SELECT COUNT(*) FROM sample) as total_samples,
    (SELECT COUNT(*) FROM age_determination) as total_ages,
    (SELECT COUNT(*) FROM age_determination WHERE NOT is_rejected) as valid_ages,
    (SELECT MIN(COALESCE(age_bp, c14_age)) FROM age_determination 
     WHERE NOT is_rejected AND COALESCE(age_bp, c14_age) >= 0) as youngest_age,
    (SELECT MAX(COALESCE(age_bp, c14_age)) FROM age_determination 
     WHERE NOT is_rejected) as oldest_age
"""

summary = query_to_df(summary_sql).iloc[0]

print("=" * 55)
print("           AUSTARCH DATABASE SUMMARY")
print("=" * 55)
print(f"  Total Sites:               {summary['total_sites']:>10,}")
print(f"  Sites with Coordinates:    {summary['sites_with_coords']:>10,}")
print(f"  Total Samples:             {summary['total_samples']:>10,}")
print(f"  Total Age Determinations:  {summary['total_ages']:>10,}")
print(f"  Valid (non-rejected):      {summary['valid_ages']:>10,}")
print(f"  Age Range:          {summary['youngest_age']:,.0f} - {summary['oldest_age']:,.0f} years BP")
print("=" * 55)

---
## Output Files

This notebook generates the following files:

| File | Description |
|------|-------------|
| `temporal_distribution.png` | Stacked bar chart of ages by time period |
| `method_breakdown.png` | Bar and pie charts of dating methods |
| `state_coverage.png` | Sites and dates by state/territory |
| `material_distribution.png` | Sample materials used for dating |
| `austarch_map.html` | Interactive map of site locations |

---

## License

This notebook is released under the MIT License. See [LICENSE](LICENSE) for details.

## Citation

If you use this visualization in your research, please cite the original AustArch dataset:

> Williams, A.N., Ulm, S., Cook, A.R., Langley, M.C. and Collard, M. (2013) AustArch: A Database of 14C and Non-14C Ages from Archaeological Sites in Australia. [https://doi.org/10.5284/1027216](https://doi.org/10.5284/1027216)